In [277]:
import yfinance as yf
import pandas as pd
import numpy as np

In [288]:
# If you don't have csv
# Download it using 
# df = yf.Ticker('reliance.NS').history(period='max', interval="1wk",actions=False)
# df.to_csv("reliance_weekly.csv")
df = pd.read_csv("reliance_weekly.csv")
df.reset_index()
df.set_index('Date')

,Open,High,Low,Close,Volume
Date,,,,,
1996-01-01,10.471229,11.081625,10.249034,10.338423,3.995653e+08
1996-01-08,10.317989,10.325650,9.283635,9.582449,7.553281e+08
1996-01-15,9.554357,9.602882,8.655363,8.714105,6.548208e+08
1996-01-22,8.742197,8.977161,7.802341,7.850866,6.486725e+08
1996-01-29,7.815112,10.522307,7.766586,10.409933,1.802410e+09
...,...,...,...,...,...
2022-04-18,2533.000000,2802.000000,2521.800049,2758.800049,4.520440e+07
2022-04-25,2742.399902,2856.149902,2663.649902,2790.250000,4.266970e+07
2022-05-02,2762.000000,2805.500000,2593.550049,2620.649902,3.026298e+07


# Methodology
I will be identifying demand zone structure and supply zone structure on basis of candlestick structure. After identifying demand zone, we can further optimize it by checking its strength, identifying factors such as consolidation and can use it to automate and make our chart analysis process easier.

Boring Candles -> Candles which match the criterion -> Size of body less than 50% of size of wicks
Exciting Candles -> Candles which match the criterion -> Size of body greater than 50% of size of wicks.

Demand zones -> Structure which satisfies this order of candlesticks in given order
-> Red Exciting Candle followed by (1-4) boring candles followed with Strong Green Exciting Candle

![Drop Base Rally Demand Zone](./images/dbr_zone.png)

In [289]:
df['boringCandle'] = (abs(df['Open']-df['Close']) / abs(df['High']-df['Low']) < 0.5)
df['greenExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']<df['Close'])
df['redExcitingCandle'] = (abs(df['Open']-df['Close'])/abs(df['High']-df['Low']) > 0.5) & (df['Open']>df['Close'])

In [290]:
df['ds_dbr_base1'] = df['redExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']
df['ds_rbr_base1'] = df['greenExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']
df['ds_dbr_base2'] = df['redExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']
df['ds_rbr_base2'] = df['greenExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['greenExcitingCandle']

df['ss_rbd_base1'] = df['greenExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']
df['ss_dbd_base1'] = df['redExcitingCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']
df['ss_rbd_base2'] = df['greenExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']
df['ss_dbd_base2'] = df['redExcitingCandle'].shift(3) & df['boringCandle'].shift(2) & df['boringCandle'].shift(1) & df['redExcitingCandle']

In [291]:
df_ds_dbr_base1 = df.iloc[df.iloc[np.where(df['ds_dbr_base1'] == True)].index - 1]
df_ds_rbr_base1 = df.iloc[df.iloc[np.where(df['ds_rbr_base1'] == True)].index - 1]

df_ss_rbd_base1 = df.iloc[df.iloc[np.where(df['ss_rbd_base1'] == True)].index - 1]
df_ss_dbd_base1 = df.iloc[df.iloc[np.where(df['ss_dbd_base1'] == True)].index - 1]

df_ds_dbr_base2_boringCandle1 = df.iloc[df.iloc[np.where(df['ds_dbr_base2'] == True)].index - 2]
df_ds_dbr_base2_boringCandle2 = df.iloc[df.iloc[np.where(df['ds_dbr_base2'] == True)].index - 1]
df_ds_rbr_base2_boringCandle1 = df.iloc[df.iloc[np.where(df['ds_rbr_base2'] == True)].index - 2]
df_ds_rbr_base2_boringCandle2 = df.iloc[df.iloc[np.where(df['ds_rbr_base2'] == True)].index - 1]

df_ss_rbd_base2_boringCandle1 = df.iloc[df.iloc[np.where(df['ss_rbd_base2'] == True)].index - 2]
df_ss_rbd_base2_boringCandle2 = df.iloc[df.iloc[np.where(df['ss_rbd_base2'] == True)].index - 1]
df_ss_dbd_base2_boringCandle1 = df.iloc[df.iloc[np.where(df['ss_dbd_base2'] == True)].index - 2]
df_ss_dbd_base2_boringCandle2 = df.iloc[df.iloc[np.where(df['ss_dbd_base2'] == True)].index - 1]

In [292]:
df_ds_dbr_base1['greenBoringCandle'] = df_ds_dbr_base1['Open'] < df_ds_dbr_base1['Close']
df_ds_rbr_base1['greenBoringCandle'] = df_ds_rbr_base1['Open'] < df_ds_rbr_base1['Close']
df_ds_dbr_base2_boringCandle1['greenBoringCandle'] = df_ds_dbr_base2_boringCandle1['Open'] < df_ds_dbr_base2_boringCandle1['Close']
df_ds_dbr_base2_boringCandle2['greenBoringCandle'] = df_ds_dbr_base2_boringCandle2['Open'] < df_ds_dbr_base2_boringCandle2['Close']
df_ds_rbr_base2_boringCandle1['greenBoringCandle'] = df_ds_rbr_base2_boringCandle1['Open'] < df_ds_rbr_base2_boringCandle1['Close']
df_ds_rbr_base2_boringCandle2['greenBoringCandle'] = df_ds_rbr_base2_boringCandle2['Open'] < df_ds_rbr_base2_boringCandle2['Close']

df_ss_rbd_base1['greenBoringCandle'] = df_ss_rbd_base1['Open'] < df_ss_rbd_base1['Close']
df_ss_dbd_base1['greenBoringCandle'] = df_ss_dbd_base1['Open'] < df_ss_dbd_base1['Close']
df_ss_rbd_base2_boringCandle1['greenBoringCandle'] = df_ss_rbd_base2_boringCandle1['Open'] < df_ss_rbd_base2_boringCandle1['Close']
df_ss_dbd_base2_boringCandle2['greenBoringCandle'] = df_ss_dbd_base2_boringCandle2['Open'] < df_ss_dbd_base2_boringCandle2['Close']
df_ss_rbd_base2_boringCandle1['greenBoringCandle'] = df_ss_rbd_base2_boringCandle1['Open'] < df_ss_rbd_base2_boringCandle1['Close']
df_ss_dbd_base2_boringCandle2['greenBoringCandle'] = df_ss_dbd_base2_boringCandle2['Open'] < df_ss_dbd_base2_boringCandle2['Close']

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/2576367893.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/2576367893.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/2576367893.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [293]:

"""
Plotting Base 1 Demand Zones
"""
dbrProximalLine = []
dbrDistalLine = []
rbrProximalLine = []
rbrDistalLine = []
for i in range(0,len(df_ds_dbr_base1)):
	df_x = df_ds_dbr_base1.iloc[i]
	if df_x['greenBoringCandle']:
		dbrProximalLine.append(df_x['Close'])
	else:
		dbrProximalLine.append(df_x['Open'])
	dbrDistalLine.append(df_x['Low'])
for i in range(0,len(df_ds_rbr_base1)):
	df_x = df_ds_rbr_base1.iloc[i]
	if df_x['greenBoringCandle']:
		rbrProximalLine.append(df_x['Close'])
	else:
		rbrProximalLine.append(df_x['Open'])
	rbrDistalLine.append(df_x['Low'])
df_ds_dbr_base1['proximalLine'] = dbrProximalLine
df_ds_dbr_base1['distalLine'] = dbrDistalLine
df_ds_rbr_base1['proximalLine'] = rbrProximalLine
df_ds_rbr_base1['distalLine'] = rbrDistalLine

rbdProximalLine = []
rbdDistalLine = []
dbdProximalLine = []
dbdDistalLine = []
for i in range(0,len(df_ss_rbd_base1)):
	df_x = df_ss_rbd_base1.iloc[i]
	if df_x['greenBoringCandle']:
		rbdProximalLine.append(df_x['Close'])
	else:
		rbdProximalLine.append(df_x['Open'])
	rbdDistalLine.append(df_x['Low'])
for i in range(0,len(df_ss_dbd_base1)):
	df_x = df_ss_dbd_base1.iloc[i]
	if df_x['greenBoringCandle']:
		dbdProximalLine.append(df_x['Close'])
	else:
		dbdProximalLine.append(df_x['Open'])
	dbdDistalLine.append(df_x['Low'])
df_ss_rbd_base1['proximalLine'] = rbdProximalLine
df_ss_rbd_base1['distalLine'] = rbdDistalLine
df_ss_dbd_base1['proximalLine'] = dbdProximalLine
df_ss_dbd_base1['distalLine'] = dbdDistalLine


C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3071710261.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3071710261.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3071710261.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [294]:
df_ss_rbd_base1_list = df_ss_rbd_base1.values.tolist()
df_ss_dbd_base1_list = df_ss_dbd_base1.values.tolist()
df_ss_rbd_base2_boringCandle1_list = df_ss_rbd_base2_boringCandle1.values.tolist()
df_ss_dbd_base2_boringCandle1_list = df_ss_dbd_base2_boringCandle1.values.tolist()
df_ss_rbd_base1_list = df_ss_rbd_base1_list[::-1]
df_ss_dbd_base1_list = df_ss_dbd_base1_list[::-1]
df_ss_rbd_base2_boringCandle1_list = df_ss_rbd_base2_boringCandle1_list[::-1]
df_ss_dbd_base2_boringCandle1_list = df_ss_dbd_base2_boringCandle1_list[::-1]
zoneTested = []
df.Date = pd.to_datetime(df.Date)
linesToDraw = []
for dz in df_ss_rbd_base1_list:
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
df_ds_dbr_base1['zoneTested'] = zoneTested
zoneTested = []	
for dz in df_ss_dbd_base1_list:
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
df_ds_rbr_base1['zoneTested'] = zoneTested
zoneTested = []	
for i in range(0,len(df_ss_rbd_base2_boringCandle1_list)):
	dz = df_ds_dbr_base2_boringCandle1.iloc[i]
	dz2 = df_ds_dbr_base2_boringCandle2.iloc[i]
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz2[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		print("base 2 rbd")
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
zoneTested = []	
for i in range(0,len(df_ss_dbd_base2_boringCandle1_list)):
	dz = df_ds_rbr_base2_boringCandle1.iloc[i]
	dz2 = df_ds_rbr_base2_boringCandle2.iloc[i]
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz2[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		print("base 2 dbd")
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)

ValueError: Length of values (35) does not match length of index (29)

In [295]:
"""
Plotting Base 2 Demand Zones
"""
dbrProximalLine2 = []
dbrDistalLine2 = []
rbrProximalLine2 = []
rbrDistalLine2 = []
for i in range(0,len(df_ds_dbr_base2_boringCandle1)):
	df_x_1 = df_ds_dbr_base2_boringCandle1.iloc[i]
	df_x_2 = df_ds_dbr_base2_boringCandle2.iloc[i]
	if df_x_1['greenBoringCandle'] and df_x_2['greenBoringCandle']:
		if df_x_1['Close'] > df_x_2['Close']:
			dbrProximalLine2.append(df_x_1['Close'])
		else:
			dbrProximalLine2.append(df_x_2['Close'])
	elif df_x_1['greenBoringCandle'] and (not df_x_2['greenBoringCandle']):
		if df_x_1['Close'] > df_x_2['Open']:
			dbrProximalLine2.append(df_x_1['Close'])
		else:
			dbrProximalLine2.append(df_x_2['Open'])
	elif (df_x_1['greenBoringCandle'] == False) and df_x_2['greenBoringCandle']:
		if df_x_1['Open'] > df_x_2['Close']:
			dbrProximalLine2.append(df_x_1['Open'])
		else:
			dbrProximalLine2.append(df_x_2['Close'])
	else:
		if df_x_1['Open'] > df_x_2['Open']:
			dbrProximalLine2.append(df_x_1['Open'])
		else:
			dbrProximalLine2.append(df_x_2['Open'])
	if int(df_x_1['Low']) < int(df_x_2['Low']):
		dbrDistalLine2.append(int(df_x_1['Low']))
	else:
		dbrDistalLine2.append(int(df_x_2['Low']))
for i in range(0,len(df_ds_rbr_base2_boringCandle1)):
	df_x_1 = df_ds_rbr_base2_boringCandle1.iloc[i]
	df_x_2 = df_ds_rbr_base2_boringCandle2.iloc[i]
	if df_x_1['greenBoringCandle'] and df_x_2['greenBoringCandle']:
		if df_x_1['Close'] > df_x_2['Close']:
			rbrProximalLine2.append(df_x_1['Close'])
		else:
			rbrProximalLine2.append(df_x_2['Close'])
	elif df_x_1['greenBoringCandle'] and (not df_x_2['greenBoringCandle']):
		if df_x_1['Close'] > df_x_2['Open']:
			rbrProximalLine2.append(df_x_1['Close'])
		else:
			rbrProximalLine2.append(df_x_2['Open'])
	elif (df_x_1['greenBoringCandle'] == False) and df_x_2['greenBoringCandle']:
		if df_x_1['Open'] > df_x_2['Close']:
			rbrProximalLine2.append(df_x_1['Open'])
		else:
			rbrProximalLine2.append(df_x_2['Close'])
	else:
		if df_x_1['Open'] > df_x_2['Open']:
			rbrProximalLine2.append(df_x_1['Open'])
		else:
			rbrProximalLine2.append(df_x_2['Open'])
	if int(df_x_1['Low']) < int(df_x_2['Low']):
		rbrDistalLine2.append(int(df_x_1['Low']))
	else:
		rbrDistalLine2.append(int(df_x_2['Low']))
df_ds_dbr_base2_boringCandle1['proximalLine'] = dbrProximalLine2
df_ds_dbr_base2_boringCandle1['distalLine'] = dbrDistalLine2
df_ds_rbr_base2_boringCandle1['proximalLine'] = rbrProximalLine2
df_ds_rbr_base2_boringCandle1['distalLine'] = rbrDistalLine2

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3701022749.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3701022749.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/3701022749.py:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [296]:
df_ss_rbd_base1_list = df_ss_rbd_base1.values.tolist()
df_ss_dbd_base1_list = df_ss_dbd_base1.values.tolist()
df_ss_rbd_base2_boringCandle1_list = df_ss_rbd_base2_boringCandle1.values.tolist()
df_ss_dbd_base2_boringCandle1_list = df_ss_dbd_base2_boringCandle1.values.tolist()
df_ss_rbd_base1_list = df_ss_rbd_base1_list[::-1]
df_ss_dbd_base1_list = df_ss_dbd_base1_list[::-1]
df_ss_rbd_base2_boringCandle1_list = df_ss_rbd_base2_boringCandle1_list[::-1]
df_ss_dbd_base2_boringCandle1_list = df_ss_dbd_base2_boringCandle1_list[::-1]
zoneTested = []
df.Date = pd.to_datetime(df.Date)
linesToDraw = []
for dz in df_ss_rbd_base1_list:
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
df_ds_dbr_base1['zoneTested'] = zoneTested
zoneTested = []	
for dz in df_ss_dbd_base1_list:
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
df_ds_rbr_base1['zoneTested'] = zoneTested
zoneTested = []	
for i in range(0,len(df_ss_rbd_base2_boringCandle1_list)):
	dz = df_ds_dbr_base2_boringCandle1.iloc[i]
	dz2 = df_ds_dbr_base2_boringCandle2.iloc[i]
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz2[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		print("base 2 rbd")
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)
zoneTested = []	
for i in range(0,len(df_ss_dbd_base2_boringCandle1_list)):
	dz = df_ds_rbr_base2_boringCandle1.iloc[i]
	dz2 = df_ds_rbr_base2_boringCandle2.iloc[i]
	proximalLine = dz[14]
	distalLine = dz[15]
	index = dz2[0]
	index = pd.to_datetime(index)
	dz_sub = df.loc[df.Date > index]
	if len(np.where(dz_sub.Low <= proximalLine)[0]) == 0:
		print("base 2 dbd")
		linesToDraw.append([index,proximalLine, distalLine])
		zoneTested.append(False)
	else:
		zoneTested.append(True)

ValueError: Length of values (35) does not match length of index (29)

In [298]:
dbrProximalLine2 = []
dbrDistalLine2 = []
rbrProximalLine2 = []
rbrDistalLine2 = []
for i in range(0,len(df_ds_dbr_base2_boringCandle1)):
	df_x_1 = df_ds_dbr_base2_boringCandle1.iloc[i]
	df_x_2 = df_ds_dbr_base2_boringCandle2.iloc[i]
	if df_x_1['greenBoringCandle'] and df_x_2['greenBoringCandle']:
		if df_x_1['Close'] > df_x_2['Close']:
			dbrProximalLine2.append(df_x_1['Close'])
		else:
			dbrProximalLine2.append(df_x_2['Close'])
	elif df_x_1['greenBoringCandle'] and (not df_x_2['greenBoringCandle']):
		if df_x_1['Close'] > df_x_2['Open']:
			dbrProximalLine2.append(df_x_1['Close'])
		else:
			dbrProximalLine2.append(df_x_2['Open'])
	elif (df_x_1['greenBoringCandle'] == False) and df_x_2['greenBoringCandle']:
		if df_x_1['Open'] > df_x_2['Close']:
			dbrProximalLine2.append(df_x_1['Open'])
		else:
			dbrProximalLine2.append(df_x_2['Close'])
	else:
		if df_x_1['Open'] > df_x_2['Open']:
			dbrProximalLine2.append(df_x_1['Open'])
		else:
			dbrProximalLine2.append(df_x_2['Open'])
	if int(df_x_1['Low']) < int(df_x_2['Low']):
		dbrDistalLine2.append(int(df_x_1['Low']))
	else:
		dbrDistalLine2.append(int(df_x_2['Low']))
for i in range(0,len(df_ds_rbr_base2_boringCandle1)):
	df_x_1 = df_ds_rbr_base2_boringCandle1.iloc[i]
	df_x_2 = df_ds_rbr_base2_boringCandle2.iloc[i]
	if df_x_1['greenBoringCandle'] and df_x_2['greenBoringCandle']:
		if df_x_1['Close'] > df_x_2['Close']:
			rbrProximalLine2.append(df_x_1['Close'])
		else:
			rbrProximalLine2.append(df_x_2['Close'])
	elif df_x_1['greenBoringCandle'] and (not df_x_2['greenBoringCandle']):
		if df_x_1['Close'] > df_x_2['Open']:
			rbrProximalLine2.append(df_x_1['Close'])
		else:
			rbrProximalLine2.append(df_x_2['Open'])
	elif (df_x_1['greenBoringCandle'] == False) and df_x_2['greenBoringCandle']:
		if df_x_1['Open'] > df_x_2['Close']:
			rbrProximalLine2.append(df_x_1['Open'])
		else:
			rbrProximalLine2.append(df_x_2['Close'])
	else:
		if df_x_1['Open'] > df_x_2['Open']:
			rbrProximalLine2.append(df_x_1['Open'])
		else:
			rbrProximalLine2.append(df_x_2['Open'])
	if int(df_x_1['Low']) < int(df_x_2['Low']):
		rbrDistalLine2.append(int(df_x_1['Low']))
	else:
		rbrDistalLine2.append(int(df_x_2['Low']))
df_ds_dbr_base2_boringCandle1['proximalLine'] = dbrProximalLine2
df_ds_dbr_base2_boringCandle1['distalLine'] = dbrDistalLine2
df_ds_rbr_base2_boringCandle1['proximalLine'] = rbrProximalLine2
df_ds_rbr_base2_boringCandle1['distalLine'] = rbrDistalLine2

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/31282704.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/31282704.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp/ipykernel_2252/31282704.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [299]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Candlestick(x=df['Date'], open=df['Open'], close=df['Close'], low=df['Low'], high=df['High']))
for row in linesToDraw:
	fig.add_shape(type="rect",line=dict(color="green", width=2),x0=row[0], y0=row[1], x1=df.Date.max(), y1=row[2], fillcolor='green', opacity=0.5)
fig.update_layout(margin={"t":25, "b":0, "l":0, "r":2})
fig.write_html("output/reliance_monthly.html")